<a href="https://colab.research.google.com/github/SDS-AAU/dsba-cbs/blob/master/M3/workshop/GPT2_textgeneration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install simpletransformers

In [1]:
import pandas as pd


df = pd.read_json("https://www.dropbox.com/s/twxfxdln1l5gm3o/tweets_11-06-2020.zip?dl=1", compression='zip')
df = df[df['device'] == 'Twitter for iPhone']
df = df[df['isRetweet'] == 'f']

In [2]:
tweets = df["text"].tolist()

In [4]:
with open("train.txt", "w") as f:
    for tweet in tweets[:-10]:
        f.writelines(tweet + "\n")

with open("test.txt", "w") as f:
    for tweet in tweets[-10:]:
        f.writelines(tweet + "\n")

In [5]:
from simpletransformers.language_modeling import LanguageModelingModel
import logging


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 32,
    "num_train_epochs": 1,
    "save_eval_checkpoints": False,
    "mlm": False,
    #"best_model_dir": "/content/drive/MyDrive/Colab/GPR2Trump/best" <-- after mounting google drive
}

model = LanguageModelingModel('gpt2', 'gpt2', args=train_args)

model.train_model("train.txt", eval_file="test.txt")

INFO:filelock:Lock 140234751020112 acquired on cache_dir/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


INFO:filelock:Lock 140234751020112 released on cache_dir/684fe667923972fb57f6b4dcb61a3c92763ad89882f3da5da9866baf14f2d60f.c7ed1f96aac49e745788faa77ba0a26a392643a50bb388b9c04ff469e555241f.lock


INFO:filelock:Lock 140234750426752 acquired on cache_dir/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 140234750426752 released on cache_dir/c0c761a63004025aeadd530c4c27b860ec4ecbe8a00531233de21d865a402598.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b.lock


INFO:filelock:Lock 140234749972608 acquired on cache_dir/fc674cd6907b4c9e933cb42d67662436b89fa9540a1f40d7c919d0109289ad01.7d2e0efa5ca20cef4fb199382111e9d3ad96fd77b849e1d4bed13a66e1336f51.lock


...

INFO:simpletransformers.language_modeling.language_modeling_model: Training started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


INFO:simpletransformers.language_modeling.language_modeling_model: Training of gpt2 model complete. Saved to outputs/.


(174, 3.726340173304766)

In [6]:
model.eval_model("test.txt")

INFO:simpletransformers.language_modeling.language_modeling_utils: Creating features from dataset file at cache_dir/


INFO:simpletransformers.language_modeling.language_modeling_utils: Saving features into cached file cache_dir/gpt2_cached_lm_126_test.txt


INFO:simpletransformers.language_modeling.language_modeling_model:{'eval_loss': 3.378248929977417, 'perplexity': tensor(29.3194)}


{'eval_loss': 3.378248929977417, 'perplexity': tensor(29.3194)}

In [7]:
# Saving the quantized model
model.save_model("/content/GPR2Trump", model=model.model)

# Load the saved (quantized) model
#model = ClassificationModel('roberta', "outputs/quantized", use_cuda=False) <-- load trained model with another model type

In [8]:
import logging
from simpletransformers.language_generation import LanguageGenerationModel


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


model = LanguageGenerationModel('gpt2', "/content/GPR2Trump", args={"max_length": 100})

In [9]:
tweets[:10][1]

'The Unsolicited Mail In Ballot Scam is a major threat to our Democracy, &amp; the Democrats know it. Almost all recent elections using this system, even though much smaller &amp;  with far fewer Ballots to count, have ended up being a disaster. Large numbers of missing Ballots &amp; Fraud!'

In [11]:
model.generate('The Unsolicited Mail In Ballot')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
INFO:simpletransformers.language_generation.language_generation_model:=== GENERATED SEQUENCE 1 ===
INFO:simpletransformers.language_generation.language_generation_model:The Unsolicited Mail In Ballot Against Trump, Is a Shill for Hillary Clinton! https://t.co/VgO1F2kS7RIt is only fair that Hillary Clinton, who has been saying she was against Sanders for years, now say she was against Trump and only wants to see one party replace Sanders, a loser! #MAGA! https://t.co/VgO1F2kS7Rhttps://t.co/L8jL7dg7S


['The Unsolicited Mail In Ballot Against Trump, Is a Shill for Hillary Clinton! https://t.co/VgO1F2kS7RIt is only fair that Hillary Clinton, who has been saying she was against Sanders for years, now say she was against Trump and only wants to see one party replace Sanders, a loser! #MAGA! https://t.co/VgO1F2kS7Rhttps://t.co/L8jL7dg7S']